In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map

In [ ]:
if torch.cuda.is_available():  # Fallback to CUDA if available
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("No GPU available, falling back to CPU.")

hw = 256
H = hw
W = hw
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [ ]:
from diff3f import VERTEX_GPU_LIMIT, batch_render
import random

num_views = 100

mesh = convert_mesh_container_to_torch_mesh(source_mesh, device=device, is_tosca=False)
mesh_vertices = mesh.verts_list()[0]

if mesh_vertices is None:
    mesh_vertices = mesh.verts_list()[0]

if len(mesh_vertices) > VERTEX_GPU_LIMIT:
    samples = random.sample(range(len(mesh_vertices)), 10000)
    maximal_distance = torch.cdist(mesh_vertices[samples], mesh_vertices[samples]).max()
else:
    maximal_distance = torch.cdist(mesh_vertices, mesh_vertices).max()  # .cpu()

ball_drop_radius = maximal_distance * tolerance

batched_renderings, normal_batched_renderings, camera, depth = batch_render(
    device, mesh, mesh_vertices, num_views, H, W, use_normal_map,
    fixed_angle={'type': 'azimuth', 'value': 0}
)

print("Rendering complete")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_rotation_sequence(batched_renderings, num_views, save_video=False):
    # Move to CPU and convert to NumPy
    batched_renderings = batched_renderings.cpu().numpy()
    
    if save_video:
        import cv2
        # Ensure the directory exists
        output_path = 'rotation_video.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 30.0, (W, H))
    
    # Visualize each frame
    for i in range(num_views):
        view = batched_renderings[i]
        
        if save_video:
            # Convert to BGR for OpenCV
            frame = (view * 255).astype(np.uint8)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            out.write(frame)
        
        # Display every 10th frame to check progression
        #if i % 10 == 0:
        plt.figure(figsize=(8, 8))
        plt.imshow(view)
        plt.title(f"Elevation: {(i/num_views)*360:.1f}°")
        plt.axis("off")
        plt.show()
        plt.close()

    if save_video:
        out.release()
        print(f"Video saved to {output_path}")

# Use the visualization function
visualize_rotation_sequence(batched_renderings, num_views, save_video=True)